## 🔢 Description Crawler

In [ ]:
# Imports
import numpy    as np
import pandas   as pd
import datetime
import tempfile
import subprocess
import requests
import time
import json
import gzip
import os

#### Initialization

In [ ]:
print("⚡ Start - {} ⚡\n".format(datetime.datetime.now()))
startTime = datetime.datetime.now()

In [ ]:
# import pandas as pd
# from langdetect import detect, DetectorFactory

# # To ensure reproducibility of language detection
# DetectorFactory.seed = 0

# # Load the data from a CSV file
# df = pd.read_csv('allDescriptions.csv')

# # Print the size before dropping duplicates
# print(f"Size before dropping duplicates: {df.shape}")

# # Drop duplicates based on the 'pkgName' column
# df_dedup = df.drop_duplicates(subset='pkgName')

# # Print the size after dropping duplicates
# print(f"Size after dropping duplicates: {df_dedup.shape}")

# # Function to check if a description is in English
# def is_english(text):
#     try:
#         return detect(text) == 'en'
#     except:
#         return False

# # Apply the function to filter out non-English descriptions
# df_filtered = df_dedup[df_dedup['description'].apply(is_english)]

# # Print the size after removing non-English descriptions
# print(f"Size after removing non-English descriptions: {df_filtered.shape}")

# # Save the cleaned DataFrame to a new CSV file
# df_filtered.to_csv('cleaned_file.csv', index=False)

#### 📥 1) Load Data 

In [ ]:
# Load the dataframes
az16DF = pd.read_csv("../../0_Data/azFiltered16.csv")
# Select and rename columns
az16DF = az16DF[['sha256', 'pkg_name']]
az16DF.rename(columns={'pkg_name': 'pkgName'}, inplace=True)

# Add new columns with specified values
az16DF['source'] = 'AZ'
az16DF['BenjaminClass'] = np.nan

az16DF

In [ ]:
# Load the data from the CSV file
mudflowDF = pd.read_csv("../../0_Data/4_MudFlow.csv")

# Select the necessary columns
mudflowDF = mudflowDF[['sha256', 'pkgName', 'classID']]

# Rename 'classID' to 'BenjaminClass'
mudflowDF.rename(columns={'classID': 'BenjaminClass'}, inplace=True)

# Add a new column 'Source' with the value 'MUDFLOW'
mudflowDF['source'] = 'MUDFLOW'

mudflowDF

In [ ]:
# Concatenate the dataframes
maliciousDF = pd.concat([az16DF, mudflowDF], axis=0)

# Print sizes of dataframes with emojis
print(f"📄 Size of az16DF      : {az16DF.shape}")
print(f"📄 Size of mudflowDF   : {mudflowDF.shape}")
print(f"📄 Size of maliciousDF : {maliciousDF.shape}")

# Remove duplicates based on sha256 column
maliciousDF = maliciousDF.drop_duplicates(subset='sha256')

# Print size of maliciousDF after removing duplicates
print(f"📄 Size of maliciousDF : {maliciousDF.shape}")

maliciousDF

In [ ]:
desDF = pd.read_csv("../../0_Data/allDescriptionsCleaned.csv")
desDF

In [ ]:
maldesDF = pd.merge(maliciousDF, desDF, on='pkgName', how='inner')

# Drop rows where 'description' column is NaN or empty
maldesDF = maldesDF[maldesDF['description'].notna() & (maldesDF['description'] != '')]

maldesDF = maldesDF.drop_duplicates(subset='sha256')

maldesDF

In [ ]:
maldesDF.to_csv("5_NewMalCatSet.csv", index=False)

##### 🔚 End

In [ ]:
endTime = datetime.datetime.now()
print("\n🔚 --- End - {} --- 🔚".format(endTime))

# Assuming endTime and startTime are in seconds
totalTime = endTime - startTime
minutes = totalTime.total_seconds() // 60
seconds = totalTime.total_seconds() % 60
print("⏱️ --- Time: {:02d} minutes and {:02d} seconds --- ⏱️".format(int(minutes), int(seconds)))